<a href="https://colab.research.google.com/github/ArjunNarayan2066/CS484_project/blob/fixed_training_loop/cs484_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install dependencies
! nvcc --version

! pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install tensorboardX==1.4
! pip install opencv-python==3.3.1.11

# Clone repo
! git clone https://github.com/ArjunNarayan2066/monodepth2.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Cloning into 'monodepth2'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 164 (delta 2), reused 7 (delta 2), pack-reused 150
Receiving objects: 100% (164/164), 10.27 MiB | 39.10 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [11]:
import torch
! uname -a
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

Linux 28dd3f85ea1d 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux
True
Tesla T4


In [ ]:
# ! python monodepth2/train.py --model_name stereo_model  --frame_ids 0 --use_stereo --split eigen_full

In [3]:
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_calib.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0001/2011_09_28_drive_0001_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0002/2011_09_28_drive_0002_sync.zip



--2020-12-18 04:00:17--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_calib.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.199
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [application/zip]
Saving to: ‘2011_09_28_calib.zip’

2011_09_28_calib.zi 100%[===================>]   3.98K  --.-KB/s    in 0s      

2020-12-18 04:00:18 (182 MB/s) - ‘2011_09_28_calib.zip’ saved [4073/4073]

--2020-12-18 04:00:18--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0001/2011_09_28_drive_0001_sync.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.199
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 424412944 (405M) [application/zip]


In [4]:
! mkdir kitti_data
! unzip -q 2011_09_28_drive_0001_sync.zip -d kitti_data
! rm -rf 2011_09_28_drive_0001_sync.zip
! unzip -q 2011_09_28_drive_0002_sync.zip -d kitti_data
! rm -rf 2011_09_28_drive_0002_sync.zip
! unzip -q 2011_09_28_calib.zip -d kitti_data
! rm -rf 2011_09_28_calib.zip

In [ ]:
! sudo apt update
! sudo apt install imagemagick
! sudo apt install parallel
! find kitti_data/2011_09_28 -name '*.png' | parallel 'convert -quality 92 -sampling-factor 2x2,1x1,1x1 {.}.png {.}.jpg && rm {}'

In [ ]:
# ! python monodepth2/export_gt_depth.py --data_path kitti_data --split eigen

In [ ]:
# ! python monodepth2/evaluate_depth.py --data_path kitti_data --load_weights_folder /root/tmp/S_640x192/models/weights_19/ --eval_stereo

In [9]:

import os
import torchvision.models as models

def save_model(train):
    """Save model weights to disk
    """
    save_folder = os.path.join("/content/test_model/")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for model_name, model in train.models.items():
        save_path = os.path.join(save_folder, "{}.pth".format(model_name))
        to_save = model.state_dict()
        if 'encoder' in model_name:
            # save the sizes - these are needed at prediction time
            to_save['height'] = train.h
            to_save['width'] = train.w
            to_save['use_stereo'] = True
        torch.save(to_save, save_path)

    save_path = os.path.join(save_folder, "{}.pth".format("adam"))
    torch.save(train.adam_optim.state_dict(), save_path)

In [8]:
#@title Default title text

###
###
### Training code is based on a very-stripped down version of https://arxiv.org/pdf/1806.01260.pdf
### Written only with stereo based training with reduced feature set
###
###


import monodepth2.layers, monodepth2.utils, monodepth2.trainer
import monodepth2.networks

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import timeit
import PIL.Image as pil


class MyTraining(object):
    def __init__(self, batch):
        self.batch = batch
        self.h = 192
        self.w = 640
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")

        # For listing which frames to check
        # For simple stereo is only current (0) and estimated pair (s)
        self.frames = [0, 's']

        # For multi-scale estimation
        self.loss_scales = [0, 1, 2, 3]

        # Weighting for loss function
        self.alpha = 0.85

        # Define projections for pose estimation
        self.depth_projection = {}
        self.projection_3d = {}
        for loss_scale in self.loss_scales:
            h = int(self.h / (2 ** loss_scale))
            w = int(self.w / (2 ** loss_scale))

            self.depth_projection[loss_scale] = monodepth2.layers.BackprojectDepth(self.batch, h, w).to(self.device)
            self.projection_3d[loss_scale] = monodepth2.layers.Project3D(self.batch, h, w).to(self.device)
        
        ## Declare Depth Network
        self.depth_encoder_network = monodepth2.networks.ResnetEncoder(18, True).to(self.device)
        self.depth_decoder_network = monodepth2.networks.DepthDecoder(self.depth_encoder_network.num_ch_enc, 
                                                [0, 1, 2, 3]).to(self.device) 

        ## Declare Pose Network
        self.pose_encoder_network = monodepth2.networks.ResnetEncoder(18, True, num_input_images=2).to(self.device)
        self.pose_decoder_network = monodepth2.networks.PoseDecoder(self.pose_encoder_network.num_ch_enc, 
                                                num_input_features=1, num_frames_to_predict_for=2).to(self.device)

        self.models = {"encoder": self.depth_encoder_network, "depth": self.depth_decoder_network,
                       "pose_encoder": self.pose_encoder_network, "pose": self.pose_decoder_network}

        # Set up our excerpt of Kitti Dataset
        training_files = monodepth2.utils.readlines("/content/monodepth2/splits/eigen_full/train_files.txt")
        validation_files = monodepth2.utils.readlines("/content/monodepth2/splits/eigen_full/val_files.txt")

        train_set = monodepth2.datasets.kitti_dataset.KITTIRAWDataset("/content/kitti_data", training_files, self.h, self.w, self.frames, 4, is_train=True, img_ext='.jpg')
        self.train_loader = DataLoader(train_set, self.batch, True, num_workers=6, pin_memory=True, drop_last=True)
        val_set = monodepth2.datasets.kitti_dataset.KITTIRAWDataset("/content/kitti_data", validation_files, self.h, self.w, self.frames, 4, is_train=False, img_ext='.jpg')
        self.val_loader = DataLoader(val_set, self.batch, True, num_workers=6, pin_memory=True, drop_last=True)

    def run_training_loop(self):
        self.all_params = list(self.depth_encoder_network.parameters())
        self.all_params += list(self.depth_decoder_network.parameters())
        self.all_params += list(self.pose_encoder_network.parameters())
        self.all_params += list(self.pose_decoder_network.parameters())

        # Same learning rate configuration as https://arxiv.org/pdf/1806.01260.pdf
        self.adam_optim = optim.Adam(self.all_params, 1e-4)
        self.lr_sched = optim.lr_scheduler.StepLR(self.adam_optim, 15, 0.1)
        self.ssim_loss_func = monodepth2.layers.SSIM().to(self.device)

        self.epoch_count = 0
        self.losses  = []

        for self.epoch_count in range(5):
            # Per Epoch
            self.lr_sched.step()
            self.depth_encoder_network.train()
            self.depth_decoder_network.train()
            self.pose_encoder_network.train()
            self.pose_decoder_network.train()

            for idx, inputs in enumerate(self.train_loader):
                # Push to GPU
                for key, ipt in inputs.items():
                    inputs[key] = ipt.to(self.device)

                # Per Batch Code
                batch_start_time = timeit.default_timer()

                feature_identifications = self.depth_encoder_network(inputs["color_aug", 0, 0])
                outputs = self.depth_decoder_network(feature_identifications)

                self.estimate_stereo_predictions(inputs, outputs)
                loss = self.batch_loss_func(inputs, outputs)

                self.adam_optim.zero_grad()
                loss["loss"].backward()
                self.losses.append(loss["loss"].item())
                self.adam_optim.step()

                batch_duration = timeit.default_timer() - batch_start_time

                # Do something with batch duration and losses
            
            print("Finished Epoch: {} with Loss {}".format(self.epoch_count, self.losses[-1]))

    def estimate_stereo_predictions(self, inputs, outputs):
        # Generate estimated stereo pair using the pose networks
        for loss_scale in self.loss_scales:
            estimated_disparity = outputs[("disp", loss_scale)]
            disp = F.interpolate(estimated_disparity, [self.h, self.w], mode="bilinear", align_corners=False)
            base = 0 # base scale

            # Convert sigmoid disparity to depth estimate
            scaled_disp = 0.001 + (10 - 0.001) * disp
            depth = 1 / scaled_disp
            outputs[("depth", 0, loss_scale)] = depth

            # Finalize Stereo Estimates
            # Fetch camera extrinsics generated by KITTIRAWDataset
            extrinsics = inputs["stereo_T"]

            camera_coords = self.depth_projection[base](depth, inputs[("inv_K", base)])
            pixel_coords = self.projection_3d[base](camera_coords, inputs[("K", base)], extrinsics)

            outputs[("sample", 's', loss_scale)] = pixel_coords
            outputs[("color", 's', loss_scale)] = F.grid_sample(
                inputs[("color", 's', base)], outputs[("sample", 's', loss_scale)], padding_mode="border")
                
    def reprojection(self, prediction, target):
        ssim_loss = self.ssim_loss_func(prediction, target).mean(1, True)
        reproj_losses = self.alpha*ssim_loss + (1-self.alpha)*(torch.abs(target-prediction).mean(1, True))
        return reproj_losses

    def batch_loss_func(self, inputs, outputs):
        batch_loss = {}
        complete_losses = 0

        for loss_scale in self.loss_scales:
            reproj_losses = []

            reproj_losses.append(self.reprojection(outputs[("color", 's', loss_scale)], inputs[("color", 0, 0)]))
            reproj_losses = torch.cat(reproj_losses, 1)

            identity_loss = []
            identity_loss.append(self.reprojection(inputs[("color", 's', 0)], inputs[("color", 0, 0)]))
            identity_loss = torch.cat(identity_loss, 1)


            # Add some minor noise to ensure no repeated values
            identity_loss += torch.rand(identity_loss.shape).cuda() * 0.00001
            total = torch.cat((identity_loss, reproj_losses), dim=1)
            val, idxs = torch.min(total, dim=1)
            outputs["identity_selection/{}".format(loss_scale)] = (idxs > identity_loss.shape[1] - 1).float()

            current_loss = val.mean()

            mean_disp = outputs[("disp", loss_scale)].mean(2, True).mean(3, True)
            norm_disp = outputs[("disp", loss_scale)] / (mean_disp + 1e-7)
            smooth_loss = monodepth2.layers.get_smooth_loss(norm_disp, inputs[("color", 0, loss_scale)])

            current_loss += 1e-3 * smooth_loss / (2 ** loss_scale)
            complete_losses += current_loss
            batch_loss["loss/{}".format(loss_scale)] = current_loss

        complete_losses /= len(self.loss_scales)
        batch_loss["loss"] = complete_losses
        return batch_loss


In [10]:
train = MyTraining(20)
train.run_training_loop()
save_model(train)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


Finished Epoch: 0 with Loss 0.14371290802955627
Finished Epoch: 1 with Loss 0.1369514912366867
Finished Epoch: 2 with Loss 0.1367199420928955
Finished Epoch: 3 with Loss 0.11680222302675247
Finished Epoch: 4 with Loss 0.12205266207456589


In [12]:
# Simple test on car image
! python monodepth2/test_simple.py --image_path monodepth2/assets/test_image.jpg --model_name mono+stereo_640x192 --model_path /content/test_model/

-> Loading model from  /content/test_model/
   Loading pretrained encoder
   Loading pretrained decoder
-> Predicting on 1 test images
   Processed 1 of 1 images - saved prediction to monodepth2/assets/test_image_disp.jpeg
-> Done!


In [7]:
import matplotlib.pyplot as plt

plt.plot(range(len(train.losses)), train.losses, 'b-')
plt.show()


AttributeError: ignored